# Data Fundamentals (H)
John H. Williamson -- Session 2024/2025

----



---

## Lab 6/7: **Assessed**
# Optimisation and gradient descent
### (V13)



$$\newcommand{\vec}[1]{ {\bf #1}} 
\newcommand{\real}{\mathbb{R}}
\DeclareMathOperator*{\argmin}{arg\,min}
\vec{x}\real
$$

## Purpose of this lab
This lab should help you:    

* understand how optimisation can be used to solve approximation problems.
* understand how hyperparameter search and penalisation are used in optimisation
* use optimisation to solve a simple machine learning problem.
* use automatic differentiation to accelerate optimisation.
* understand the carbon impacts of machine learning.

 <div class="alert alert-info">
    
**This exercise is structured more like a tutorial, with more reading and experimentation than writing new code.**
    
In the lecture notes I used `History` to track states of an optimiser. You will not have this class available in this lab; you will have to track the states of the optimisation yourself. Do **not** try and use `History` in your solution.
    
</div>    


You will need to use these functions in the latter part of the lab:

* `autograd.grad` to compute gradients.
* `autograd.flatten` to convert structured data to flat vectors.

You will implement a very simple form of **deep learning** in this lab, using first-order optimisation to learn an approximating function.
If you find the concepts difficult, you might find this video helpful: [**How machines learn**](https://www.youtube.com/watch?v=IHZwWFHWa-w).

---

You will need to write functions that return functions in this lab (closures), like the example below. If you've not done this before, look at the example below (or search for "python closures").

In [ ]:
def make_fn(attribute, fruit_1):
    def describe_fruit(fruit_2, comparison):
        # note: attribute and fruit_1 are bound
        # into this function -- it remembers them
        print(f"{fruit_1}'s {attribute} is {comparison} than {fruit_2}")
    return describe_fruit

apple_taste = make_fn("taste", "apple") # makes a new function
grape_shape = make_fn("shape", "grape") # makes a new function
apple_taste("banana", "sweeter") # calls the new function
apple_taste("hedgehog", "better") # calls the new function
grape_shape("pineapple", "smoother")

## Before you start
You need to install `autograd.` The cell below will autoinstall this for you if the machine you are using does not already have it installed. **Remember to restart the kernel after packages are installed, and run this cell again.**

In [ ]:
from proxy_install import proxy_install

proxy_install("autograd")
proxy_install("dyn_plot")
proxy_install("jhwutils")

import autograd.numpy as np
rng = np.random.default_rng(12345)
from autograd import grad 
from autograd.misc.flatten import flatten    

# custom utils
from jhwutils.checkarr import array_hash, check_hash
import jhwutils.image_audio as ia
import jhwutils.tick as tick
np.set_printoptions(suppress=True)
import warnings
warnings.filterwarnings('ignore')

# Set up Matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
print("✅ Everything imported OK!")
plt.rc('figure', figsize=(8.0, 4.0), dpi=140)

## Convergence graphs

It's important to be able to debug how an optimisation is going. We'll use a very simple `Convergence` graph to plot the loss function as we go along. Run the example below for how it works. If you can't get the graph to work on your machine, you can call `Convergence(graph=False)` to switch to a text update.

**Use Convergence() in all of your solutions (the skeleton code will usually include it for you). *Always* show the objective function's value somehow!**

In [ ]:
import time 
from convergence import Convergence, no_graph

# try Convergence(graph=False) to see the difference
# interval=1 means it will update the plot every iteration
# (you can set e.g. interval=100 to update every 100 iterations)
# you can always change the interval to see more or less clearly 
# (but it will slow down the code a bit if you make it too frequent)
c_graph = Convergence(interval=1)

target = 310
guess = 0.9 

while abs(guess**2 - target)>0.0001:
    guess = (guess + target/guess)/2
    # just to slow it down a bit so you can see
    # don't do this in your code!
    time.sleep(0.1)     
    c_graph.update(abs(guess**2 - target))
c_graph.close() # optional, just removes extra plot
 

# A: Doing the washing 


You are in charge of a new machine learning startup to improve the way clothes washing is done. Automatic washing machines are one of the most important labour saving devices ever invented, and freed up people (mainly women) to do more interesting things with their lives than scrubbing dirt out of fabric. However, basic washing machines are imperfect; they consume vast quantities of energy (and thus generate large CO2 emissions) as well as fresh water, and increase pollution through generation of grey water and microplastics.

How can we use **optimisation** to improve this?


<img src="imgs/laundry.jpg" width="40%">

*[Image credit: Toby Bochan CC-BY-NC 2.0, https://www.flickr.com/photos/tobyleah/359877499]*



### A.1 Random search for bedding 

A laundry engineer has come up with a complicated mathematical model to indicate how well bedsheets get washed for a particular combination of soap concentration and temperature, on a scale from 0-10, where 10 is best. The formula is given by the code `bedding_model` below:


In [ ]:
def bedding_model(params):
    # some crazy model of how a washing machine works
    soap_pct, temp = params

    def sm(x):
        return 1 / (1 + np.exp(-x))

    s = (
        sm((temp - 55.0) / 5.0)
        + np.exp(-((soap_pct - 1.0) ** 2) * 9.0)
        + 3 * sm((temp * soap_pct - 21.0) / 15.0)
    )
    s = (
        s
        + 4
        * np.exp(-((soap_pct ** (temp * 0.015) - 1.5) ** 2) * 1.1)
        * np.exp(-(((temp - 10) / ((soap_pct**2 + 0.212) / 4.0) - 82.5) ** 2) / 400)
        ** 0.5
    )
    return s * sm((temp - 20.0) / 13.0) * 1.3 * (1 - sm((temp - 90.0) / 5.0))

In [ ]:
# show the results for a few example soap percentages
fig, ax = plt.subplots()
soap_pct = np.linspace(0, 3, 100)
temp = np.linspace(10, 100, 100)

for i, soap_pct in enumerate([0.0, 0.25, 0.5, 1, 2, 3]):
    ls = [":", "--", "-.", "-", "--", ":"]
    ax.plot(
        temp, bedding_model([soap_pct, temp]), label=f"soap={soap_pct}%", ls=ls[i], lw=1
    )
ax.legend()
ax.set_xlabel("Temperature (°C)")
ax.set_ylabel("Wash quality")
plt.show()

What is the best setting for soap_pct and temperature to get the best wash quality?

One simple optimisation solution would be to use **random search**, iteratively evaluating $L(\theta)$ where $\theta$ = `[soap_pct, temperature]`. 

We just need to make random guesses in some feasible set and keep the best solution found so far. Write code that does this, calling your function `random_search(obj_fun, ranges, max_iter, rng)` that takes:

* an objective function `obj_fun(theta)` that takes a parameter vector $\theta$ and returns a scalar value, 
* an array `ranges` of shape $(N, 2)$ specifying the minimum and maximum value for $\theta$ for each dimension (i.e. the feasible set),
* runs for `max_iter` iterations.
* It should take a random number generator `rng` as an argument. Use *uniform* random numbers (`rng.uniform`) to generate the random values. 

Return the best $\theta$ found.

**Write a general random_search function -- do not hardcode bedding_model! Use Convergence to show your progress**

Once you have a working random search, use it to find the best washing machine `temperature` and `soap_pct` and store them in the variables `best_temperature` and `best_soap_pct`. 

* Remember that optimisation functions always *minimise* their objective function. 
* Assume the feasible set for the washing machine is $[0, 5]$ for soap_pct and $[0, 100]$ for temperature.
* You should find a good setting within 20_000 iterations.


In [ ]:
def random_search(l, ranges, max_iter, rng):
    conv = Convergence(interval=500) 
    # YOUR CODE HERE
    conv.close()
    return best_theta
    

In [ ]:
rng = np.random.default_rng(12345)

def simple_objective_function(theta):
    return (5-(theta[0]-theta[1])**2)**2

def hard_objective_function(theta):    
    return np.sum((np.argsort(theta)-theta)**2)

def test_function(theta):
    return 1

# check that the general random search function works
with tick.marks(4):    
    print("Testing random search...")
    with no_graph():
        rng = np.random.default_rng(12345)
        test_theta = random_search(test_function, np.array([[-5,5]]).reshape(-1, 2), 1, rng)        
        assert np.allclose(test_theta, [-2.7266397], atol=1e-1), "Are you using the passed in rng? And just making one guess per iteration?"        
        rng = np.random.default_rng(12345)
        best_simple_theta = random_search(simple_objective_function, np.array([-5,5]*2).reshape(2, 2), 750, rng)
        best_simple_loss = simple_objective_function(best_simple_theta)
        assert best_simple_loss < 0.1, "Are you you sure you are *minimising* the objective function?"
        rng = np.random.default_rng(12345)
        best_hard_theta = random_search(hard_objective_function, np.array([0,5]*8).reshape(-1, 2), 10000, rng)
        best_hard_loss = simple_objective_function(best_hard_theta)
        assert best_hard_loss < 20,  "Hmm, something is off with random search. Check your ranges and number of iterations." 

In [ ]:
rng = np.random.default_rng(12345)
# YOUR CODE HERE

In [ ]:
with tick.marks(2):
    assert abs(best_soap_pct - 1.5) < 0.3
    assert abs(best_temperature - 66.0) < 3
    assert bedding_model([best_soap_pct, best_temperature]) > 9.5

## A.2 Enzymatic curves 

Biological washing powders use *enzymes* to break down contaminants in the wash. The activity of these enzymes is a function of the pH level (acidity) of the water. Biologists have done extensive experiments and found experimentally activity levels of enzymes across a range of pHs.

We can see this collection of points, each representing one input $x$ (pH) and one output $y$ (enyzmatic activity). A washing machine might want to be able to use a model like this to predict how to release enzymes based on the current pH to get the best wash. But we can't just store hundreds of data points in the microcontroller of a washing machine. We need to find a simple *function* that approximates this data that could be stored in a small memory.


In [ ]:
# the raw data we are going to plot
ph_data = np.loadtxt("data/ph_data.csv", delimiter=",") 
ph_data[:,0] = (ph_data[:,0]-7)/7

fig, ax = plt.subplots()
ax.scatter(ph_data[:,0]*7+7, ph_data[:,1], s=2)
ax.set_xlabel("pH")
ax.set_ylabel("Enzymatic activity")
ax.set_title("pH vs Enzymatic activity")
ax.set_xlim(0,14)
plt.show()

### Approximation
Imagine I wanted to approximate a smooth, positive curve from this data (e.g. so I could *interpolate* -- predict in between data points). How might I do it? I might decide on a curve form that is easy to compute. One simple kind of curve is given by the equation:

$$f(x) = \log \left(1+e^ {\left(\sum_{i=1}^n a_i (x-a_0)^i e^{-i}\right)}\right)\\
$$

> Note: you **absolutely do not** have to understand this function, but it's a simple curve that can approximate a lot of other (positive) curves.

This has a bunch of parameters $a_{0}, a_{1}, ..., a_{n-1}$ that define the curve. Adjusting those values adjusts the curve shape. If I fix some maximum number of terms $n$, I can write this function down in code, packing all of these values into one vector $\theta=[a_0, a_1, ..., a_{n-1}]$.:

In [ ]:
def poly_predict(x, theta):
    """
    Given an array of inputs x, and a theta = [a0, a1, a2, ...],
    return an array of predictions for each x using the formula above
    """    
    y = np.zeros_like(x)    
    # just the equation in code
    for i, a_i in enumerate(theta[1:]): 
        y += a_i * (x-theta[0]) ** i * np.exp(-i)           
    return np.log(1+np.exp(y))



We can show the result with some arbitrary choices for $\theta$:

In [ ]:
def show_curve(pts, theta, title=""):
    fig, ax = plt.subplots()
    # create points over the input range
    xs = np.linspace(-1, 1, 200)
    # predict the output for those points
    y_pred = poly_predict(xs, theta)
    ax.plot(xs*7+7, y_pred, label="Predicted curve")
    ax.scatter(pts[:,0]*7+7, pts[:,1], label="Data", c='C1', s=2)
    ax.legend()
    ax.set_xlabel('input')
    ax.set_ylabel('output')
    ax.set_xlim(0, 14)
    ax.set_ylim(0.0, 1.5)
    ax.set_title(title)

# random theta, bad fit
theta = [0, 0.5, 0, -1, 2]

show_curve(pts=ph_data, theta=theta)
plt.show()

Write down a function that would measure how good the fit of a function to a set of points is using the arithmetic mean of squared errors (differences between true and predicted). The function `mean_squared(y, y_pred)` should take:

* true inputs `y`
* predicted outputs `y_pred`

This is a simple form of a **loss function**. The loss function is a measure of how well the model is doing.

In [ ]:
# YOUR CODE HERE

In [ ]:
with tick.marks(2):
    assert mean_squared(np.array([1,2,3]), np.array([1,2,3])) == 0
    assert mean_squared(np.array([1,2,3]), np.array([4,5,6])) == 9
    assert mean_squared(np.array([3,2,1]), np.array([4,5,6])) == 11.66666666666666667

We could measure how good a particular $\theta$ is by measuring how much error there was predicting the $y'=f(x)$ of the data points from the inputs $x$ (each of which is an $x,y$ pair) using this loss function. 

We want to find the $\theta$ that minimises this loss function. Write a function `approximation_loss` that takes:

* a set of data points `x` and `y`
* a loss function `loss_fn`
* a prediction function `f`

and returns a **new function** that takes *only* a parameter vector $\theta$ and returns the loss. **Note: you need to use a nested function or a lambda expression to do this.**

In [ ]:
def approximation_loss(x, y, f, loss):
    # YOUR CODE HERE
    return loss_fn     

In [ ]:
with tick.marks(2):
    my_loss = approximation_loss(ph_data[:,0], ph_data[:,1], poly_predict, mean_squared)
    assert callable(my_loss), "You need to return a function"
    stupid_loss = approximation_loss(np.array([1,2,3]), np.array([1,2,3]), lambda x,theta: x, mean_squared)
    assert stupid_loss([]) == 0, "You need to use the loss function"
    stupid_loss_2 = approximation_loss(np.array([1,2,3]), np.array([4,5,6]), lambda x,theta: x, mean_squared)
    assert stupid_loss_2([]) == 9, "You need to use the loss function"
    test_loss = approximation_loss(np.array([1,2,3]), np.array([4,5,6]), lambda x,theta: x+theta, mean_squared)
    assert test_loss([3,3,0]) == 3, "Are you calling f?"

Now we could find a curve that fits our data by minimising this loss function using `random_search`. Find a curve using a $\theta$ of length 4 (i.e. $\theta=[a_0, a_1, a_2, a_3]$) and store the best parameters as `best_theta`. 

Use a range of [-5, 5] for the range of guess for each of the entries in $\theta$.

Note: the result will not be very good. Use a max of 10000 iterations to find the best $\theta$.

In [ ]:
rng = np.random.default_rng(12345)
# YOUR CODE HERE

In [ ]:
show_curve(pts=ph_data, theta=best_theta, title="Fitted curve")
with tick.marks(4):
    loss = approximation_loss(ph_data[:,0], ph_data[:,1], poly_predict, mean_squared)(best_theta)
    print(loss)
    assert loss < 0.3, "Are you sure you are minimising the loss?"

## A.3 Hill-climbing

Random search doesn't assume that parameters are continuous-valued or that the objective function is smooth. Therefore, it is not a particularly good optimisation algorithm for problems where these assumptions hold. Stochastic hill-climbing allows for a more efficient search by making "local" movements in parameter space.

The code below implements hill-climbing for you:

In [ ]:
def get_init_fn(limits, rng):
    # this function returns a function init_fn with a fixed range
    return lambda: rng.uniform(limits[:,0], limits[:,1])

def get_proposal_fn(delta, rng):    
    # this function returns a function proposal_fn with a fixed delta
    return lambda x: x+rng.normal(0, delta, x.shape)
    
def hill_climbing(l, max_iters, init_fn, proposal_fn):    
    best_guess = init_fn()    
    best_loss = l(best_guess) # work out how bad it is    
    c = Convergence(interval=1000) # you can change this <--
    for i in range(max_iters):                
        proposed = proposal_fn(best_guess)   
        proposed_loss = l(proposed)
        # check if we beat the record
        if proposed_loss<best_loss:
            best_guess, best_loss = proposed, proposed_loss            
        c.update(best_loss)
    c.close()
    return best_guess

# example usage
# dimensions = 6
# limits = [-5, 5]
# delta = 0.1
# hill_climbing(my_loss, 10_000, get_init_fn(limits=np.array([limits]*dimensions), rng=rng), get_proposal_fn(delta=delta, rng=rng))

Apply hill-climbing to the problem of finding the best curve to fit the data, using a **8 dimensional** $\theta$ (again, all values initially in range [-5, 5]). Use a maximum of 20_000 iterations. You will need to adjust the hyperparameter `delta` (step size) to get good results. Store the best $\theta$ in the variable `best_theta` and the best loss in the variable `best_loss`.

In [ ]:
rng = np.random.default_rng(12345)
# YOUR CODE HERE

In [ ]:
show_curve(ph_data, best_theta, title=f"Hill climbing: Loss {best_loss:.2f}")
plt.show()

with tick.marks(4):
    assert approximation_loss(ph_data[:,0], ph_data[:,1], poly_predict, mean_squared)(best_theta) == best_loss
    assert best_loss < 0.04, "Hmm, you might need to adjust delta or the number of iterations"  

## A.4 Annealing the motor parameters


<img src="imgs/motor.jpg" width=30%>

*[Image credit: Hausmann, https://www.deviantart.com/hausmann/art/Electric-Motor-Final-Cross-section-665887814, CC-NC-ND-3.0]* 

You are asked to help optimise the *motors* inside a washing machine. These motors have 50 distinct parameters that can be adjusted to change the timing of the way motor elements are energised. The motors are very sensitive to the exact settings of these parameters, and the engineers have found that the motors work poorly if the settings are not just right. Unfortunately, there are 50 parameters and all of the parameters interact in a complex way.

Fitting a smooth curve to points is relatively easy in optimisation terms; the resulting objective function is smooth and **convex**. This means that there is only one minimum, and it is easy to find by "walking down the hill". But nonconvex functions like this particular motor problem have many minima. It has many "pockets" which will trap local optimisers, and in high dimensions, it is very hard to find the global minimum just by guessing.

A 2D slice of the "motor parameter" objective function that the engineers are trying to optimise is shown below:

In [ ]:
# nasty objective function
def motor_model(x):
    x = x - np.arange(x.shape[-1])*0.5
    return  np.linalg.norm((x**2 - 19*np.cos(1.5*x+np.sin(x+1.2))), axis=-1, ord=2) 
grid = np.linspace(-7,7, 120)
mx, my = np.meshgrid(grid, grid)
mz = motor_model(np.stack([mx, my]).T)

fig, ax = plt.subplots()
ax.contourf(mx, my, mz, levels=25, cmap='magma')
ax.set_xlabel("Motor param 1")
ax.set_ylabel("Motor param 2")
ax.set_aspect(1.0)
plt.show()

# show it in 3D
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d', proj_type = 'persp')
ax.set_xlabel("Motor param 1")
ax.set_ylabel("Motor param 2")


ax.plot_surface(mx, my, mz, color='w', rstride=2, cstride=2)
# adjust elevation
ax.view_init(50, 40)


Neither hill-climbing nor random search will be able to find good solutions:

In [ ]:
# try and solve it in 50 dimensions with hill climbing
# it will plateau at a local minimum
rng = np.random.default_rng(12345)

init_fn = get_init_fn(limits=np.array([-50,50]*50).reshape(-1, 2), rng=rng)
proposal_fn = get_proposal_fn(delta=0.2, rng=rng)
theta = hill_climbing(motor_model, 30_000, init_fn, proposal_fn)

In [ ]:
# also very ineffective, dimension is too high
rng = np.random.default_rng(12345)
theta = random_search(motor_model, np.array([-50,50]*50).reshape(-1, 2), 30_000, rng)

**Task**

Create a modified version of `hill_climbing` called `annealed_climbing` which implements **simulated annealing**. 

In simulated annealing, a temperature $T$ is used to control the probability of accepting a worse solution (i.e. an uphill step). 

* Instead of only accepting steps that go "downhill", the probability of accepting a new step is given by $p = \exp(-\Delta E / T)$
* $\Delta E$ is the difference in objective function's value between the current and proposed parameter vectors. 
* The temperature $T$ starts at a defined level $T_0$ is reduced on each iteration by a factor $\alpha$, where $\alpha = 1-e^{-T_d}$.
* You can make a decision with probability $p$ by generating a uniform random number $r$ in [0,1] and accepting if $r < p$.

Use `annealed_climbing` to find a good setting for the motor problem in the true 50D space (the code in the test below will do this for you, if you implement `annealed_climbing` correctly). 

**You will need to experiment to choose a good value for the hyperparameters $T_d$ and $T_0$ to get good results. $T_d$ should be in the range [1, 20], and $T_0$ should be in the range [1, 5000]**

In [ ]:
def annealed_climbing(l,  max_iters, init_fn, proposal_fn, T_d, T_0, rng):    
    
    c = Convergence(interval=1000)   
    alpha = 1 - np.exp(-T_d)
    T = T_0
    # YOUR CODE HERE
    c.close()
    return best_guess

# Change these
# T_d, T_0 = 50, 0.5

# YOUR CODE HERE

In [ ]:
with tick.marks(4):
    rng = np.random.default_rng(12345)
    
    init_fn = get_init_fn(limits=np.array([-50,50]*50).reshape(-1, 2), rng=rng)
    proposal_fn = get_proposal_fn(delta=0.22, rng=rng)    

    with no_graph():        
        theta = annealed_climbing(motor_model, 4_000, init_fn, proposal_fn, T_d=1000, T_0=1000000.0, rng=rng)    
        assert motor_model(theta) > 500.0
    
    # this code will search for a solution to the motor problem in 50D
    theta = annealed_climbing(motor_model, 30_000, init_fn, proposal_fn, T_d=T_d, T_0=T_0, rng=rng)
    assert motor_model(theta) < 150.0 and motor_model(theta) > 0.1
    
print(f"Achieved loss of {motor_model(theta):.2f}")

## A.5 The environmental cost of machine learning

Washing is one of the main contributors to CO2 production. **Approximately 20% of the UK's total CO2 emissions arise from heating water for washing clothes**. 

The [ONS](https://www.theguardian.com/environment/2024/sep/21/uk-public-washing-their-clothes-too-often-says-major-laundry-brand#:~:text=Figures%20from%20the%20Office%20for,260%20wash%20loads%20a%20year.) reports that the UK washes approximately 2.6 billion loads of washing per year. An intelligent washing machine could radically reduce the temperature to get equally clean clothes, for example by using optimised enzyme release timing or more efficient motor control.

However, machine learning is itself a significant contributor to CO2 emissions. The training of machine learning models is a computationally intensive process that can consume vast amounts of energy. How would we quantify how much optimisation is worth doing to bring down the CO2 emissions of washing? (this is a genuinely important question to understand!)



<img src="imgs/coal_plants.jpg"> 

*[Photo by: Robert S. Donovan (CC-BY-NC 2.0)]*

---

The hyperparamater optimisation we've seen is a bit slow, but at least you can run it on a standard laptop. Production machine learning models can consume vast amounts of computation during hyperparameter optimisation. It is not uncommon for a single training run to take weeks of computation on clusters of tens of thousands of GPUs. Running the computational infrastructure contributes to CO2 emissions and thereby accelerates global heating. 

**How can we quantify the amount of CO2 emitted during machine learning model training? One estimation and comparison tool is provided at [https://mlco2.github.io/impact/](https://mlco2.github.io/impact/).**

Use the link provided above to estimate the kg CO2 emitted during optimisation under the following assumptions. These assumptions are realistic for big ML training runs undertaken by companies like Google, Facebook, and Amazon.

---
* hyperparameter optimisation is carried out using **grid search**
* one optimisation run takes 2 days on 8 GPU cluster of RTXA6000s
* 4096 optimisation runs are needed to complete a hyperparameter grid search of 4 parameters divided into 8 values each (4096 = $8^4$)
* `Amazon Web Services` is used as the cloud provider
* Compute instances are based in the `Canada (Central)` region
* *ignore any reported offsetting*
---

* Store the estimated CO2 emissions for this hyperparameter optimisation process in `co2_canada`

---


* Evaluate the CO2 impact of performing this computation instead with `CoreWeave` in the `US Central` region (store as `co2_coreweave`) and store the increase in CO2 from AWS Canada to this service in `co2_diff`.

--- 

* Assume one tree absorbs 22 kg of CO2 per year (this is extremely approximate -- there's not a "standard tree"!). How many **more** trees would need to be planted to offset the CO2 emissions from the optimisation when run on CoreWeave's service than on AWS Canada? Store this in `wasted_trees`

--- 

* Assume you have four hyperparameters and you apply **grid search** to find the best hyperparameter settings. For the AWS Canada run above, if you instead had seven hyperparameters (again divided into eight values each, and again on the AWS Canada server) how many **more** trees would need to be planted to offset the increase CO2 emissions from the finer grid search? Store this in `wasted_trees_grid`. Assume nothing else affects the time taken to run the grid search.

In [ ]:
# YOUR CODE HERE

In [ ]:
with tick.marks(2):
    assert(check_hash(np.array(co2_canada, dtype=int), ((), 47185.0)))
    print(f"The Canada compute run would have produced {co2_canada} kg of CO2")

In [ ]:
with tick.marks(2):
    print(f"The CoreWeave (US) compute run would have produced {co2_coreweave:.0f} kg of CO2")
    assert(check_hash(np.array(co2_coreweave, dtype=int), ((), 990900.0  )))    
    assert(check_hash(np.array(wasted_trees, dtype=float), ((), 42896.2909090909  )))
    print(f"You would need an extra {wasted_trees:.0f} trees to absorb this much carbon compared to AWS (Canada), approximately {wasted_trees/30e6:.3%} of Scotland's total annual plantation.")
    

In [ ]:
with tick.marks(2):    
    print(f"The move from four to seven hyperparameters in the grid search would take {wasted_trees_grid:.0f} trees to absorb the extra carbon, approximately {wasted_trees_grid/30e6:.3%} of Scotland's total annual plantation.")
    assert(check_hash(np.array(wasted_trees_grid, dtype=float), ((), 1095999.7681818183)))

The table below shows the CO2 cost of running a load of washing in a standard washing machine at different temperatures. Imagine we are using the hyperparameter optimisation to find the best settings for a new washing machine that reduces the temperature needed to get a "good" wash. 

Assume the following facts:

* Someone has calculated how much expected reduction in temperature we can expect from each hyperparameter evaluation made (of course, we don't usually have good estimates of this, but let's pretend we do). The function `estimated_temp_reduction(hyperparam_evals)` (below) estimates the average reduction in washing temperature that could be expected after running `hyperparam_evals` evaluations of hyperparameters.
* 2.6 billion washes are done per year in the UK.
* The average temperature of a wash is currently 41.2C.
* The optimisation evaluations are done on AWS Canada.

---

What is the maximum number of hyperparameter evaluations that would be viable to reduce carbon emissions in the UK over the course of a year? Specifically, what is the largest number of optimization runs that could be performed where the CO2 emissions from these evaluations remain lower than the annual CO2 emissions saved by the improved average temperature reduction?

Store the answer in `viable_runs`. Round to the nearest 1000.

How many divisions of the *four hyperparameter* grid search  does this correspond to (rounding to the nearest integer)? Store this in `viable_divisions`.

*Note: this is an optimisation problem! Hint: The objective function has the form: minimise |f(a)-g(a)|. Make sure you start with initial guesses in a suitable range (e.g. billions of hyperparameter runs)*

*If you get stuck, try plotting a graph to see the shape of the objective function before optimising*

In [ ]:
# Usage of CO2
# Note: it's nearly impossible to find exact numbers for all of these
# but this is pretty close to the average CO2 equivalent consumption 
# for a standard UK 5kg washing machine load (at least for 30,40,60,90)

def washing_co2(temp):
    return 0.1 + ((temp - 20)/82.0 + (temp-20)**2/6000.0) 

def estimated_temp_reduction(hyperparams_evals):
     def sigmoid(x):
          return 1/(1+np.exp(-x))
     return sigmoid(hyperparams_evals/1e5)*0.25 + np.exp(-hyperparams_evals-10)

# show graphs of the CO2 usage and the estimated temperature reduction
washing_temperature_c = np.array([20, 30, 40, 50, 60, 70, 80, 90])
co2_usage_kg = washing_co2(washing_temperature_c)

fig, ax = plt.subplots()
ax.plot(washing_temperature_c, co2_usage_kg, label="CO2 usage (kg)")
ax.set_xlabel("Washing temperature (°C)")
ax.set_ylabel("CO2 usage (kg)")
ax.set_title("CO2 usage vs Washing temperature per load")
ax.set_ylim(0, 2)
ax.set_xlim(20, 90)

fig, ax = plt.subplots()
hypers = np.linspace(0,1_000_000,100)
ax.plot(hypers, estimated_temp_reduction(hypers), label="Estimated reduction in temperature")
ax.set_xlabel("Hyperparameter evaluations")
ax.set_ylabel("Estimated temperature reduction (°C)")
ax.set_title("Estimated temperature reduction vs Hyperparameter evaluations")
plt.show()

In [ ]:
rng = np.random.default_rng(12345)

# YOUR CODE HERE


In [ ]:
print(f"It would be worth doing up to {viable_runs:.0f} optimisations, or {viable_divisions:.0f} divisions of the 4 parameter grid search.")
with tick.marks(6):
    assert check_hash(np.array(viable_runs), ((), 27110000.0   )), "Make sure you are minimising differences. Also, you may have converged to a local minimum; try larger initial guesses"
    assert check_hash(np.array(viable_divisions), ((), 240.0 ))
    

## A.6 Gradients and penalisation

**Note: this part uses ideas from Week 7. You can do the first parts of Part B without Week 7, if you want to skip this part for now.**

Let's return to optimising the wash temperature and soap concentration. The function `extended_bedding_model` below is an extended model of linen washing that includes two extra parameters: how long the wash runs for, and how much enzymatic content there is in the soap.

We can do much better than the random search we used in part A.1, which won't work well in four dimensions. We can use **gradient descent** to find the best solution. The `autograd` package gives us a function called `grad(f)` that takes a function `f` and returns a new function that computes the gradient of `f` with respect to its first argument. 

Use this to implement gradient descent: $$\theta_{t+1} = \theta_t - \delta \nabla L(\theta_t)$$

where $\delta$ is the step size (also called the "learning rate").  Again, write a *general* function `gradient_descent(obj_fun, init_theta, max_iter, delta)` that takes:

* an objective function `obj_fun(theta)` that takes a parameter vector $\theta$ and returns a scalar value,
* an initial guess `init_theta`,
* runs for `max_iter` iterations,
* a learning rate `delta`.

Return the best $\theta$ found (as in all of the optimisers).

Estimate the best washing parameters, and store the result as `best_wash_params`. Start with a fixed guess of $\theta=[1, 30, 10, 5]$ and use a learning rate of 0.1. 

**Note: if you get `nan` you are either maximising where you should be minimising, or you have too large a learning rate.**

In [ ]:
def extended_bedding_model(params):
    # some crazy model of how a washing machine works
    def sm(x):
        return 1 / (1 + np.exp(-x))

    soap_pct, temp, run_time, enzyme = params 
    soap_pct = soap_pct ** 2
    enzyme_activity = np.exp(-(temp-37)**2/1000.0)
    soap_pct = soap_pct + sm((enzyme_activity*enzyme-20)/20.0)    
    
    s = (
        sm((temp - 55.0) / 5.0)
        + np.exp(-((soap_pct - 1.0) ** 2) * 9.0)
        + 3 * sm((temp * soap_pct  - 21.0) / 15.0)
    )    
    s = (
        s
        + 4
        * np.exp(-((soap_pct ** np.abs(temp * 0.015) - 1.5) ** 2) * 1.1)
        * np.exp(-(((temp - 10) / ((soap_pct**2 + 0.212) / 4.0) - 82.5) ** 2) / 400)
        
    )
    s = s * sm((temp - 20.0) / 13.0) * 1.3 * (1 - sm((temp - 90.0) / 5.0))
  
    s = s * sm((run_time * temp * run_time  - 30000.0) / 9000.0)
    s += np.exp(-run_time*2.0) 
    return s

# Run this to see how it performs
fig, ax = plt.subplots(ncols=6, nrows=4, figsize=(24,12))

temp = np.linspace(10, 100, 100)
for k, enzyme in enumerate([0, 10, 20, 30]):
    for j, run_time in enumerate([0, 10, 20, 30, 40, 50]):
        for i, soap_pct in enumerate([0.0, 0.25, 0.5, 1, 2, 3]):
            ax[k,j].plot(
                temp,
                extended_bedding_model([soap_pct, temp, run_time, enzyme]),
                label=f"soap={soap_pct}%",
        )    
        ax[k,j].set_ylim(0, 10)
        ax[k,j].set_title(f"{run_time} minutes, {enzyme} enzymes")
        ax[k,j].set_xlabel("Temperature (°C)")
        ax[k,j].set_ylabel("Wash quality")
plt.tight_layout()        
plt.show()

In [ ]:
def gradient_descent(obj_fun, init_theta, max_iter, delta):
    c = Convergence(interval=1000)
    # YOUR CODE HERE
    c.close()
    return theta


best_wash_params = gradient_descent(lambda theta:-extended_bedding_model(theta), np.array([1, 30.0, 10, 5]), 20_000, 0.1)

In [ ]:
with tick.marks(2):
    # check gradient descent works 
    with no_graph():
        simple_quadratic = lambda x: (x-3)**2
        q_theta = gradient_descent(simple_quadratic, 0.0, 100, 0.1)
        assert np.allclose(q_theta, 3.0, atol=1e-2), "Are you sure you are descending the gradient?"
        assert np.abs(simple_quadratic(q_theta))<1e-2, "Are you sure you are minimising the function?"
        vector_quadratic = lambda x: np.sum((x-3)**2)
        q_theta = gradient_descent(vector_quadratic, np.array([0.0, 0.0]), 100, 0.1)
        assert np.allclose(q_theta, [3.0, 3.0], atol=1e-2), "Are you sure you are descending the gradient?"
    

In [ ]:
with tick.marks(2):
    assert extended_bedding_model(best_wash_params) > 4.5
    print(f"The best washing parameters are {best_wash_params[0]:.2f}% soap, {best_wash_params[1]:.0f}°C, {best_wash_params[2]:.0f} minutes, {best_wash_params[3]:.2f}% enzymes")

A new environmental directive has been introduced that requires that the product of the run time (in minutes) and the temperature (in degrees) in a wash cycle of any new washing machine must be less than 1600. 

Implement a penalty function that penalises parameters that do not meet this requirement. That is: $L'(\theta) = L(\theta) + \lambda C(\theta),$ where $C(\theta)$ is your penalty function. Call your penalty function `co2_penalty(theta)`.

The penalty function should return increasingly large values as the constraint is approached. Include this penalty in your objective function and use gradient descent to find a solution that meets this environmental directive. Store the result as `best_wash_params_penalty`.

In [ ]:
# YOUR CODE HERE

In [ ]:
fig, ax = plt.subplots()
for temp in [10, 20, 30, 40, 50, 60, 70, 80, 90]:
    run_time = np.linspace(0, 100, 100)
    ax.plot(run_time*temp, co2_penalty([1, temp, run_time, 1]))
    ax.set_ylim(-0.5, 10)
    ax.set_xlim(0, 3000)
    ax.set_xlabel("Product of temperature and run time")
    ax.set_ylabel("CO2 penalty")

with tick.marks(2):
    assert co2_penalty([1, 30, 10, 1]) == co2_penalty([5, 30, 10,2]), "The penalty should only depend on the product of temp and run time"
    assert co2_penalty([1, 30, 10,1 ]) < co2_penalty([1, 50, 50,1]), "Hmm, the penalty does not seem to be enforced"
    assert co2_penalty([1, 35, 50,1]) < co2_penalty([0, 50, 50,1]), "Hmm, the penalty does not seem to be enforced"

In [ ]:
with tick.marks(3):
    print(f"The best washing parameters are {best_wash_params_penalty[0]:.2f}% soap, {best_wash_params_penalty[1]:.0f}°C, {best_wash_params_penalty[2]:.0f} minutes, {best_wash_params_penalty[3]:.2f}% enzymes")
    assert extended_bedding_model(best_wash_params_penalty) > 3.3
    assert best_wash_params_penalty[1] * best_wash_params_penalty[2] < 1700, "The CO2 penalty is not being enforced well"

# B: Sorting the washing

Sorting the washing ("laundry" in American-speak) is annoying. How would we make progress towards a robot that could do it for us? 

<img src="imgs/laundry.webp" width="400px">

*[Our glorious future. Credit: DALL-E.]*

*You know that locked door at the front of the Boyd Orr lab, beside the visualiser? There really is a giant laundry sorting robot hiding in there.*

One of the first steps in building a laundry-sorting robot is to categorize different types of clothes by looking at them. How could optimization help with this? We'll need a function that takes an image of clothing and sorts it into a category based on its features. Importantly, this function should be parameterizable—meaning we can adjust its parameters to improve how it categorizes items.

#### Approximation
We will be trying to approximate a function. This means we have an objective function of the form:

$$L(\theta) = \|f(\vec{x};\theta)-y\|$$

where we measure the difference between a predicted output $f(\vec{x};\theta)$ and an expected output $y$, and try and minimise that difference by choosing a good setting for $\theta$.

We will build a simple "deep learning" system. We will completely ignore many of the important problems in machine learning, like overfitting, regularisation, efficient network architectures and fair evaluation, and concentrate on using first-order optimisation (gradient descent) to find a function that approximates this categorisation.

### Classifier 
We will be approximating a classification function -- that is, we're going to ascribe a discrete label to an input image. We will be using a simple neural network to do this.

## B.1 Prepare the data

### Loading the data
There are 4096 tiny images of articles of clothing in `data/fashion_train.png`. Each image is 28x28 pixels. The code below loads them. 

There are also a set of **labels**, which indicates the kind of article of clothing each is, as a number from 0-9.

* 0 T-shirt/top
* 1 Trouser
* 2 Pullover
* 3 Dress
* 4 Coat
* 5 Sandal
* 6 Shirt
* 7 Sneaker
* 8 Bag
* 9 Ankle boot

In [ ]:
# load the data
clothes_image = ia.load_image_colour('data/fashion_train.png')
clothes_labels = np.loadtxt('data/fashion_train_labels.txt', dtype=int)
ia.show_image(clothes_image, width="50%")
print(clothes_image.shape) # rows, columns, channels
print(clothes_labels.shape)
label_names = ["tshirt", "trouser", "pullover", "dress", "coat", "sandal", "shirt", "sneaker", "bag", "boot"]

Now, you need to transform them into a more helpful format:

* Use only the *second* color channel of the image.
* Rearrange this into a `(4096, 28, 28)` tensor. Hint: you need to reshape and then permute the axes.
* Store the result in `clothes`

In [ ]:
# YOUR CODE HERE

In [ ]:
with tick.marks(4):
    assert clothes.shape==(4096, 28, 28)
    assert np.max(clothes)<=1.0
    assert np.min(clothes)>=0.0
    assert np.mean(clothes) > 0.1
    cl = clothes.reshape(64, 64, 28, 28).swapaxes(0,1).reshape(4096, 28, 28)     
    assert not array_hash(cl) == ((4096, 28, 28), 1468618882187.8767 ), "You've got the (big) rows and columns the wrong way around"
    assert check_hash(clothes, ((4096, 28, 28),1468618882187.8767  ))

You'll see a selection of low-res images below if your code is correct.

In [ ]:
def show_cloth(cloth, ax=None):
    if ax is None:
        fig, ax = plt.subplots()
    ax.imshow(cloth, cmap='bone')
    ax.axis('off')

rng = np.random.default_rng(12345)
fig, _ = plt.subplots(4, 5, figsize=(4,3))
for i in range(20):
    ax = plt.subplot(4, 5, i+1)
    idx = rng.choice(clothes.shape[0])  
    show_cloth(clothes[idx], ax=ax)
    fig.set_facecolor('black')

## A deep network
How will this parameterisable function that maps from images to orientations be defined? We will use a very simple deep "neural network" predictor. This is an incredibly simple algorithm. It takes an input vector, then repeatedly:

* applies a fixed elementwise non-linearity; this allows the network to learn non-linear functions;
* multiplies the vector by a matrix and adds a vector. Traditionally, these are called the "weights" and "biases" of the network, but it's just $\vec{y} = W\vec{x} + \vec{b}$

We have to define the *shape* of each of the matrices which will be used to transform the vector, but we *optimise* to find the elements that go into those matrices. This is the "learning" part.

Each of these steps is traditionally called a "layer" of the prediction function.

In [ ]:
# a very basic neural network
def swish(x):
    return x * (1.0/(1.0 + np.exp(-x)))

def nn_predict(theta, x, unflatten):     
    # for each layer   
    for w,b in unflatten(theta):         
        x = w.T @ swish(x) + b
    return x

## B.2 Making vectors

We have to be able to write this problem in the form:

$$L(\theta) = \|f(\vec{x};\theta)-\vec{y}\|$$

Every input vector $\vec{x}$ must have a corresponding matched expected output $\vec{y}$, and we need a function $f$ that depends on $\vec{x}$ and $\theta$. **All of these things ($x,y,\theta$) have to be made into vectors**.

### B.2.1 Input vectors: unraveling images
What is $\vec{x}$? How can we define the input to this function? We need to have one **vector** per example; that is a matrix with one row per face image. We can do this by reshaping the `clothes` tensor to unravel the 28x28 pixel image into a single 784 dimensional vector.

**Task** 

Reshape the clothes tensor to a (4096,784) matrix; each row being a clothes image as a single "unravelled" vector. Store this in `clothes_inputs`.

In [ ]:
# YOUR CODE HERE

In [ ]:
with tick.marks(4):
    assert clothes_inputs.shape == (4096, 784)
    print(array_hash(clothes_inputs))
    assert(check_hash(clothes_inputs,((4096, 784),1468618882187.8767 )))

### B.2.2. Output vectors: one-hot encoding

The labels we have are just numbers from 0-9. We *could* try and build a function that outputs those numbers directly, but it won't work very well. It's much better to encode the labels as vectors ; this will make it much easier to learn the function that maps from inputs to outputs.

To do this, we'll use a standard transform called **one-hot encoding**. This means that each label from a set of labels of size N is represented as a vector of length N, with a 1 in the position of the label and 0 elsewhere.

For example, the label 3 (dress) in the 10 labels in our dataset would be encoded as the vector `[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]`. (remember we count from 0).

Write two functions:
* `to_one_hot(labels, max_d)` that takes a (N,) vector of labels and returns a matrix of one-hot encoded (N, max_d) labels. Max_d is the number of distinct labels (e.g. 10 for the clothes dataset).
* `from_one_hot(labels)` that takes a (N, max_d) matrix of one-hot encoded labels and returns a (N,) vector of labels. Note: do not assume that you get a perfect encoding; you should return the label with the *highest* value.

Assume that labels are consecutive integers from 0 to `max_d-1`. Note: these functions are trivial to implement in NumPy.

Then one-hot encode the `clothes_labels` into `clothes_outputs` (a `(4096,10)` matrix).


In [ ]:
# YOUR CODE HERE

In [ ]:
with tick.marks(4):
    assert to_one_hot(np.array([0]), 10).shape == (1, 10)
    assert to_one_hot(np.array([0, 1, 2]), 10).dtype == np.float64, "Make sure to convert your array to floating point with .astype(np.float64)"
    assert to_one_hot(np.array([0, 1, 2]), 10).shape == (3, 10)
    assert to_one_hot(np.array([0, 1, 2]), 3).shape == (3, 3)
    assert np.allclose(to_one_hot(np.array([0, 1, 2]), 3)[0],[1, 0, 0])
    assert np.allclose(to_one_hot(np.array([0, 1, 2]), 3)[1],[0, 1, 0])
    assert np.allclose(to_one_hot(np.array([0, 1, 2]), 3)[2],[0, 0, 1])
    assert np.allclose(from_one_hot(np.array([[1,0,0], [0,1,0], [0,0,1]])), [0, 1, 2])
    assert from_one_hot(np.array([[1,0,0,0]])) == [0]
    assert from_one_hot(np.array([[0.9,0,0,0]])) == [0], "Are you finding the highest value?"
    assert from_one_hot(np.array([[10,0,0,0]])) == [0]
    assert from_one_hot(np.array([[0.9,0.9,0.9,1]])) == [3], "Are you finding the highest value?"
    assert from_one_hot(np.array([[0,0,0,1]])) == [3]    
    assert check_hash(clothes_outputs, ((4096, 10), 83888018.4))
    rng = np.random.default_rng(12345)
    corrupted = clothes_outputs + np.random.uniform(-0.2, 0.2, clothes_outputs.shape)
    assert np.all(from_one_hot(corrupted) == clothes_labels), "Make sure you are finding the highest value"


### B.2.3. Creating theta: flattening and initial guesses

To be able to optimise this prediction function in the standard form, we have to package *all* of the things that could vary into a single "flat" parameter vector $\theta$. `nn_predict()` can unpack a list of matrices from a single vector if it is given the right `unflatten` parameter. We can use the `flatten` convenience function to make this easy.

    theta, unflatten = flatten(structure_of_matrices)

See the example below for how this works:

In [ ]:
rng = np.random.default_rng(12345)
w1 = rng.uniform(-0.1, 0.1, (784, 100))
w2 = rng.uniform(-0.1, 0.1, (100, 10))
b1 = np.zeros(100)
b2 = np.zeros(10)
# list of pairs (matrix, vector), (matrix, vector), ...
w_b = [[w1, b1], [w2, b2]]
# turn into a flat theta
flat, unflatten = flatten(w_b)
nn_predict(flat, x=clothes_inputs[0], unflatten=unflatten)

#### Network architecture
The choice of the matrix shapes we use to do the prediction affects how well we will be able to model the transformation. In our example, we know we have 784 dimensional inputs (28x28  images unraveled into flat vectors) and 10D outputs (one-hot encoded labels). The parameters of each "layer" will be composed of a matrix (the weights) and a vector (the bias).  Introducing intermediate layers allows the network to learn complicated mappings between different spaces, and not just simple linear transformations.

A very simple model might have a mapping like `784 -> 10` with just one single layer. 

A more complex model might have a mapping with extra layers: `784 -> 64 -> 32 -> 16 -> 10`. We would have weight matrices like:

    W[0]        W[1]        W[2]        W[3]
    784, 64     64, 32      32, 16      16, 10
    
(the biases `b` are not shown here, but they are also part of the $\theta$ vector). This **architecture** maps the 784 input vector to some 64 dimensional space, then to some 32 dimensional space, and so on. Every matrix *has* to have an output dimension which matches the input dimension of the following matrix. The matrices W[0], W[1], W[2] specify how the vector at each layer gets mapped to the next layer. All of these matrices `W[i]` and all of the corresponding biases `b[i]` are flattened into a single vector `theta` for optimisation. 

The choices of these matrix shapes are not hugely important (I just made the ones above up); but more matrices with more elements means a more flexible function which can learn more complicated things; but will be harder to optimise efficiently.

#### Initialisation
We need to set up an initial $\theta$ vector for our model. We can define the shape of each matrix and vector, but we don't know what the values of the elements in the matrices should be, because we will find this by optimisation. So we just make a random guess.

Create a function `initial_conditions` that:
* takes a list of layer *shapes* `[(M1,N1), (M2, N2), ...]`, and create a corresponding list of pairs (tuples) of randomly filled matrices and vectors (each M,N matrix should be paired with a length N vector).  
* The function should take a parameter `sigma` that should specify the spread of the random values chosen for each layer.
* The function should take a random number generator `rng` used to generate random values.

Use `rng.normal(0, sigma, shape)` to generate the random numbers.

Return the **flattened** version of the matrix list, along with the corresponding `unflatten` function.

In [ ]:
def initial_conditions(shape_list, sigma_list, rng):    
    # YOUR CODE HERE

In [ ]:
with tick.marks(4):
    rng = np.random.default_rng(12345)
    theta, unflatten = initial_conditions([[8,4], [4,8], [4,2]], [0.1]*3, rng)
    assert callable(unflatten), "You should return the unflatten function that flatten gives you."
    matrices = unflatten(theta)
    assert len(matrices)==3
    assert len(matrices[0])==2, "You need to return a matrix, vector pair for each layer"
    assert matrices[0][0].shape[1]==matrices[0][1].shape[0], "The shapes of the matrices need to match the vectors"
    assert(matrices[0][0].shape==(8,4))
    assert(matrices[0][1].shape==(4,))  
    assert(matrices[1][0].shape==(4,8))
    assert(matrices[1][1].shape==(8,))
    assert(matrices[2][0].shape==(4,2))
    assert(matrices[2][1].shape==(2,))
    assert np.allclose(matrices[0][0][0][0], -0.14238250364546312), "Did you use rng.normal?"

## Random predictions
We can now use `nn_predict` to see the effect of applying a random initial condition function to the clothes. Since all of the matrices are random, the result will be a random mess of very poorly sorted washing.

In [ ]:
# create some test initial conditions
rng = np.random.default_rng(1998)
n = 256
random_theta, unflatten = initial_conditions( [[784, 10]], [1, 1, 1, 1], rng)

# predict the outputs (will be random junk)
predicted_outputs = [nn_predict(random_theta, clothes_inputs[i], unflatten) for i in range(n)]
predicted_labels = from_one_hot(predicted_outputs)

def show_encoded_clothes(images, predicted_labels):
    red = np.array([1, 0, 0])
    green = np.array([0, 1, 0])
    fig = plt.figure(dpi=100, figsize=(12, 8))
    ax = fig.add_subplot(1,1,1)
    ax.set_facecolor('k')    
    racks = [0] * 10     
    for i in range(images.shape[0]): 
        x = racks[predicted_labels[i]]
        y = predicted_labels[i]
        racks[predicted_labels[i]] += 1        
        if predicted_labels[i] == clothes_labels[i]:
            img = images[i].reshape(28,28)[:,:,None] * green        
        else:
            img = images[i].reshape(28,28)[:,:,None] * red
        ax.imshow(img, extent=[x, x+1, y, y+1])
    

    ax.set_xlim(0, 80)
    ax.set_ylim(0, 10)
    ax.set_yticks(np.arange(10)+0.5)
    ax.set_yticklabels(label_names)
    ax.set_title("Encoded clothes")
    ax.set_xlabel('')
    ax.set_ylabel('Category')
    fig.set_facecolor('black')
    # set ticks/axis labels to white
    ax.xaxis.label.set_color('white')
    ax.yaxis.label.set_color('white')        
    ax.tick_params(axis='y', colors='white')
    # set the tick labels to white
    for label in ax.get_yticklabels():
        label.set_color('white')
    # set title to white
    ax.title.set_color('white')
    
    
show_encoded_clothes(clothes_inputs[:n], np.array(predicted_labels)[:n])
plt.gca().set_title("Random predictions")
plt.show()

## B.3 Objective function

### B.3.1 Per-vector loss
Write an objective function that will compare the predicted output to the actual output, given `theta`, `unflatten`, and `x` and `y`. Use $L_2$ norm as the loss function. The loss function will need to call `nn_predict` to calculate `y_prime`, the predicted output to compare with `y`. Assume `x` and `y` are vectors. 

In [ ]:
def nn_loss(theta, x, y, unflatten):
    # YOUR CODE HERE

In [ ]:
# create a random set of weights, and test that the loss function is computed correctly


with tick.marks(4):    
    theta, unflatten = flatten([])
    # check that the identity holds
    assert nn_loss(theta, np.zeros(784), np.zeros(784), unflatten) == 0.0
    assert nn_loss(theta, clothes_inputs[5],clothes_inputs[5], unflatten) == 0.0
    assert nn_loss(theta, np.zeros(784), clothes_inputs[5], unflatten) == np.linalg.norm(clothes_inputs[5]), "Are you using the L2 norm?"

    # check that doing random things gives a non-zero result
    rng = np.random.default_rng(12345)
    theta, unflatten = flatten([
        (rng.normal(0,1,(784, 100)), rng.normal(0,1,100)),
        (rng.normal(0,1,(100, 784)), rng.normal(0,1,784))
        ])
    assert nn_loss(theta, np.zeros(784), np.zeros(784), unflatten) > 0.0
    # check that different shapes for input and output work okay
    theta, unflatten = flatten([
        (rng.normal(0,1,(784, 100)), rng.normal(0,1,100)),        
        ])
    assert nn_loss(theta, np.zeros(784), np.zeros(100), unflatten) > 0.0

## B.3.2 Total loss
Write a function `reconstruction_loss` that computes the sum of the objective function value for *every* input in `clothes_inputs`.  That is, it computes:

$$L(\theta) = \sum_i \|y_i - f(\vec{x_i};\theta)\|$$

where $x_i$ is the ith row of `clothes_inputs`, $y_i$ is the corresponding row of `clothes_output` and `theta` is the parameter vector. The function must also take an `unflatten` argument so it can unpack the parameter vector correctly. **This function should use the `loss` function you defined above**

In [ ]:
# compute the sum of losses
# for every pair of xs and ys
def reconstruction_loss(theta, unflatten):
    # YOUR CODE HERE
    

In [ ]:
rng_test = np.random.default_rng(2000)

# create some test initial conditions
random_theta, random_unflatten = flatten(
    [(rng_test.normal(0,1,(784,10)), rng_test.normal(0,1,(10,)))])

with tick.marks(2):        
    print(reconstruction_loss(random_theta, random_unflatten))
    assert(np.allclose(reconstruction_loss(random_theta, random_unflatten), 84246.65556971483 , atol=1))

# B.4 Optimisation

### Bad: hillclimbing
We could try learning the `reconstruction_loss` objective function with hill climbing, but we'll find we have very poor performance and it is *extremely* slow even with a very simple network. Try the code below; it will take about 30-60 seconds to do something, and while the result is better than random it is not good.

In [ ]:
flat, unflatten = initial_conditions([[784, 10]], [0.1], rng)
make_guess = lambda: initial_conditions([[784, 10]], [0.1], rng)[0]                                        
t_loss = lambda theta:reconstruction_loss(theta, unflatten)
with no_graph():
    best_theta = hill_climbing(t_loss, 50, make_guess, get_proposal_fn(0.01, rng))
print(t_loss(best_theta))


In [ ]:
# chaotically sorted laundry
def show_predictions(theta, unflatten, n, show=True):
    predicted_outputs = [nn_predict(theta, clothes_inputs[i], unflatten) for i in range(n)]
    predicted_labels = from_one_hot(predicted_outputs)
    if show:
        show_encoded_clothes(clothes_inputs[:n], np.array(predicted_labels)[:n])
        plt.gca().set_title("Sorted clothes")
        plt.show()
    pct_correct = np.mean(predicted_labels == clothes_labels[:n])
    print(f"Correctly sorted {pct_correct:.1%} of the clothes")
    
    return pct_correct

show_predictions(best_theta, unflatten, n=1024)    
plt.show()


### B.4.1. Good: Stochastic gradient descent
Now we will optimise with stochastic gradient descent, and get a much better result.  In this task, you will have to define a function `sgd_learn`. 

`sgd_learn(obj_fun, shapes, inputs, outputs, sigmas, rng, step, iters)` should:
* take an objective function
* and a list of matrix `shapes`, a list of `sigmas` to specify the random initialisation of those matrices, a `rng` random number generator, a `step` size (delta) and a number of `iterations`
* generate an initial `theta` from that set of shapes, using the `initial_condition()` function you defined above.
* for each of the given number of iterations
    * randomly select *ONE* input vector (from `clothes_inputs`) and matching output vector (from `clothes_outputs`).
    * compute the gradient of the objective function for that image/output pair
    * make a step, adjusting `theta` in the direction of this gradient, scaled by the step size

* return the flattened theta vector and the corresponding unflatten function

You will need to compute the gradient of the objective function using `grad`. Remember that `grad(f)` takes a function `f` and returns **a new function** that computes the gradient of `f` with respect to its first argument.

Note: You don't need to implement *any* sophistications like momentum or random restart. You don't need to collect the data into minibatches. The algorithm you implement should be very simple. (you can add these things if you want, but it is not necessary to pass all of the tests).


In [ ]:
losses = []


def sgd_learn(obj_fun, shapes, inputs, outputs, sigmas, rng, step=0.1, iters=10000):
    c = Convergence(interval=100, graph=True)
    # YOUR CODE HERE
    c.close()
    return w, unflatten

In [ ]:
# verify that the shapes come out right
with tick.marks(2):
    with no_graph():
        test_theta, test_unflatten = sgd_learn(nn_loss, [[784,10]], clothes_inputs, np.ones(len(clothes_inputs)), [0.1]*3, rng, 0.1, 1)
    unflattened = test_unflatten(test_theta)
    assert(len(unflattened)==1)
    assert((unflattened[0][0].shape)==(784,10))


In [ ]:
## verify that some learning happens
rng_test = np.random.default_rng(2024)
with tick.marks(4):
    with no_graph():
        xs = clothes_inputs
        ys = np.ones(len(clothes_inputs))
        test_theta, test_unflatten = sgd_learn(nn_loss, [[784,1]], xs, ys, [0.1]*3, rng_test, 0.1, 1)
        before_losses = [nn_loss(test_theta, x,y, test_unflatten) for x,y in zip(xs, ys)]
        
        test_theta, test_unflatten = sgd_learn(nn_loss, [[784,1]], xs, ys, [0.1], rng_test, 0.05, 1000)
        after_losses = [nn_loss(test_theta, x,y, test_unflatten) for x,y in zip(xs, ys)]
        print("Mean loss before optimising %.2f; after optimising %.2f" % (np.mean(before_losses),  np.mean(after_losses)))
        assert(np.mean(after_losses)/np.mean(before_losses)<0.5)
        print("Something was learned!")
        

## B.4.2 Sort the washing

Use this function to learn the mapping function from clothes images to their label vectors.

* a **step size** (values in the range 0.2 to 0.0001 are reasonable)
* the **sigmas** for the initial conditions (values in the range 0.5 to 0.005 are reasonable).

These are **hyperparameters** of the optimisation process. 

* You should use *no more* than 50_000 iterations in the learning process. 

You should use a smaller number of iterations while you are establishing your hyperparameters, and then run the full number of iterations once you have a good idea of the best hyperparameters.

**Warning: if your call to `sgd_learn` takes much more than ten minutes to run, the autograder will not accept your result!**

Use matrix shapes `[[784,64], [64, 32],  [32,10]`, *or* choose your own set of matrix shapes (just don't make them so large the optimisation takes forever).


In [ ]:
## Run sgd_learn(...) in this cell
## produce the output theta, unflatten
## theta, unflatten = learn(...
## LEAVE THIS HERE TO FORCE CONSISTENT RESULTS!
rng = np.random.default_rng(2024)
plt.show()
# YOUR CODE HERE

In [ ]:
# let's sort the washing!
pct_correct = show_predictions(theta, unflatten, n=1024)    

In [ ]:
# you get more marks for a more correct answer :)
with tick.marks(1):
    pct_correct = show_predictions(theta, unflatten, n=1024, show=False)    
    assert pct_correct>0.2

In [ ]:
with tick.marks(2):
    pct_correct = show_predictions(theta, unflatten, n=1024, show=False)    
    assert pct_correct>0.5

In [ ]:
with tick.marks(4):
    pct_correct = show_predictions(theta, unflatten, n=1024, show=False)    
    assert pct_correct>0.75

In [ ]:
with tick.marks(2):
    pct_correct = show_predictions(theta, unflatten, n=1024, show=False)    
    assert pct_correct>0.82

----

If you've got this far, you've managed to build a system which can predict sort your washing. Well, sort of. As long as you have perfectly aligned images without any distracting background. All that was needed was some simple gradient descent, which optimised a function parameterised with a single high-dimensional vector (the number of elements in `theta`).

> Note: we didn't really test this fairly; we just tested if we could recognise the images we had trained on. A real system would test if the predictions worked on images that were not in the training set, as we may have learned irrelevant features of the training set ("overfitting"). You can test this if you want: there's a corresponding `data/fashion_test.png` and `data/fashion_test_labels.txt` file you could use to test the system. But you don't get any marks for this!

# END OF LAB

In [ ]:
# prestige marks only! This does not affect your final mark
# if you want you can try and get this
# you will need to do more than just adjust hyperparameters to do this
# (e.g. change architecture, add real batching, etc.)
with tick.marks(0):
    pct_correct = show_predictions(theta, unflatten, n=1024, show=False)    
    assert pct_correct>0.90

-----

# Submission instructions (for assessed labs only)

## Before submission

* Make sure you fill in any place that says `YOUR CODE HERE` or `"YOUR ANSWER HERE"`.
* SAVE THE NOTEBOOK
* DO NOT RENAME THE NOTEBOOK OR IT WILL NOT BE MARKED.

<div class="alert alert-block alert-danger">
    
### Formatting the submission
* **WARNING**: If you do not submit the correct file, you will not get any marks.
* Submit this file **only** on Moodle. It will be named `<xxx>.ipynb`.

</div>


## Penalties (only for assessed labs)
<font color="red">
    
**Malformatted submissions**
</font>
These assignments are processed with an automatic tool; failure to follow instructions *precisely* will lead to you automatically losing two bands in grade regardless of whether the work is correct (not to mention a long delay in getting your work back). **If you submit a file without your work in it, it will be marked and you will get 0 marks.**

<font color="red">**Late submission**</font>
Be aware that there is a two band penalty for every *day* of late submission, starting the moment of the deadline.

<font color="red">
    
**Plagiarism**
</font> Plagiarism will be subject to the Plagiarism Policy. The penalties are severe.